In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from category_encoders import BinaryEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score


font_path = '../Font/GmarketSansTTFMedium.ttf'
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)

plt.rcParams['axes.unicode_minus'] = False

In [2]:
seoul = pd.read_csv('../Data/preprocessing/seoul_real_transcation_price.csv')

In [3]:
seoul.head(5)

,시도명,시군구명,관리기관,회사명,공장구분,단지명,설립구분,입주형태,보유구분,등록구분,...,지식산업센터명,대표업종,업종명,업종코드,차수,법인주소,필지수,공장주소,공장주소_지번,공장관리번호
0,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별,,일반,해당없음,임대,신규등록,...,,18111,경 인쇄업 외 2 종,"181,111,811,318,119",10,서울특별시 종로구 자하문로16길 8 (창성동),1,서울특별시 종로구 자하문로16길 8 (창성동),서울특별시 종로구 창성동 87-1번지,1.110000e+14
1,서울특별시,종로구,서울특별시 종로구,(주)에취.알.디,개별,,일반,해당없음,임대,등록변경,...,,33932,전시용 모형 제조업,33932,10,서울특별시 종로구 통의동 35-69번지,1,서울특별시 종로구 통의동 35-69번지,서울특별시 종로구 통의동 35-69번지,1.110000e+14
2,서울특별시,종로구,서울특별시 종로구,(주)코리아쉬핑가제트,개별,,일반,해당없음,자가,등록변경,...,,58113,일반 서적 출판업 외 3 종,"58,113,181,111,811,300,000",10,서울특별시 도봉구 창동 181-17번지 쌍용아파트 115-1902,1,서울특별시 종로구 자하문로2길 13-3 (통의동),서울특별시 종로구 통의동 35-6번지,1.110000e+14
3,서울특별시,종로구,서울특별시 종로구,삼영DP,개별,,일반,해당없음,임대,신규등록,...,,18111,경 인쇄업 외 2 종,"181,111,811,318,119",10,서울특별시 종로구 사직로 125 (적선동),1,서울특별시 종로구 사직로 125 (적선동),서울특별시 종로구 적선동 107-1번지,1.110000e+14
4,서울특별시,종로구,서울특별시 종로구,삼진기획,개별,,일반,해당없음,임대,등록변경,...,,18119,기타 인쇄업 외 1 종,"1,811,918,113",10,서울특별시 서대문구 홍제동 -번지 문화촌현대아파트 103-1101호,1,서울특별시 종로구 적선동 2번지,서울특별시 종로구 적선동 2번지,1.110000e+14


In [4]:
seoul.columns

Index(['시도명', '시군구명', '관리기관', '회사명', '공장구분', '단지명', '설립구분', '입주형태', '보유구분',
       '등록구분', '전화번호', '남자종업원', '여자종업원', '외국인남자종업원', '외국인여자종업원', '종업원합계',
       '생산품', '원자재', '공장규모', '용도지역', '지목', '용지면적', '제조시설면적', '부대시설면적', '건축면적',
       '지식산업센터명', '대표업종', '업종명', '업종코드', '차수', '법인주소', '필지수', '공장주소',
       '공장주소_지번', '공장관리번호'],
      dtype='object')

### 삭제할 칼럼
시도명, 시군구명, 관리기관, 회사명, 단지명, 전화번호, 법인주소, 필지수, 공장주소, 공장관리번호, 시

### 보류 칼럼 (일단 삭제)
생산품, 원자재, 업종명, 대표업종, 차수, 필지수

### 전처리 진행
용도지역, 업종코드 값 000 제거, 공장주소 지번 -> 동으로 쪼개기

In [5]:
seoul = seoul.drop(columns=['시도명', '시군구명', '관리기관', '회사명', '단지명', '전화번호', '법인주소', '필지수', '공장주소', '공장관리번호']) #1차
seoul = seoul.drop(columns=['생산품', '원자재', '업종명', '업종코드', '차수']) #2차

In [6]:
seoul.head(3)

,공장구분,설립구분,입주형태,보유구분,등록구분,남자종업원,여자종업원,외국인남자종업원,외국인여자종업원,종업원합계,공장규모,용도지역,지목,용지면적,제조시설면적,부대시설면적,건축면적,지식산업센터명,대표업종,공장주소_지번
0,개별,일반,해당없음,임대,신규등록,0,0,0,0,0,소기업,도시지역/주거지역/제2종일반주거지역/고도지구,대,238.00,57.12,69.44,126.56,,18111,서울특별시 종로구 창성동 87-1번지
1,개별,일반,해당없음,임대,등록변경,5,2,0,0,7,소기업,도시지역/주거지역/주거지역/,대,157.55,79.11,78.44,157.55,,33932,서울특별시 종로구 통의동 35-69번지
2,개별,일반,해당없음,자가,등록변경,12,12,0,0,24,소기업,도시지역/주거지역/주거지역/,대,160.30,92.40,67.90,160.30,,58113,서울특별시 종로구 통의동 35-6번지


In [7]:
seoul['대표업종'].value_counts()

18119    1088
28123     369
14192     328
26421     309
26410     288
         ... 
20412       1
26120       1
30121       1
62010       1
13910       1
Name: 대표업종, Length: 450, dtype: int64

### 공장주소_지번을 시, 구, 동으로 구분하기

In [7]:
address = seoul['공장주소_지번'].str.split(n=2, expand=True)
address[2] = address[2].str.split().str[0] #남은 부분에서 첫번째만 분리

print(address)

address.columns = ['시', '구', '동']
seoul = pd.concat([seoul, address[['시', '구', '동']]], axis=1)
seoul = seoul.drop(columns=['공장주소_지번', '시', '동'])


           0    1    2
0      서울특별시  종로구  창성동
1      서울특별시  종로구  통의동
2      서울특별시  종로구  통의동
3      서울특별시  종로구  적선동
4      서울특별시  종로구  적선동
...      ...  ...  ...
11628  서울특별시  강동구  천호동
11629  서울특별시  강동구  천호동
11630  서울특별시  강동구  천호동
11631  서울특별시  강동구  천호동
11632  서울특별시  강동구  천호동

[11633 rows x 3 columns]


In [8]:
object_columns = ['공장구분', '설립구분', '입주형태', '보유구분', '등록구분', '공장규모', '용도지역', '지목', '지식산업센터명', '대표업종', '구']
int_columns = ['남자종업원', '여자종업원', '외국인남자종업원', '외국인여자종업원', '종업원합계','용지면적', '제조시설면적', '부대시설면적', '건축면적']

In [9]:
# object 타입 변환
for column in object_columns:
    seoul[column] = seoul[column].astype('object')

for column in int_columns:
    seoul[column] = seoul[column].astype('int')

In [10]:
seoul['지목'] = seoul['지목'].str.strip() # 공백제거
seoul['지목'].replace('', '해당없음', inplace=True)
seoul['지목'].fillna('해당없음', inplace=True)

In [11]:
seoul['지식산업센터명'] = seoul['지식산업센터명'].str.strip() # 공백제거
seoul['지식산업센터명'].replace('', '해당없음', inplace=True)
seoul['지식산업센터명'].fillna('해당없음', inplace=True)

In [12]:
import pickle
# 라벨 인코더와 오리지널 값을 저장할 딕셔너리
label_encoders = {}
original_values = {}

# 모든 object 타입의 컬럼에 대해 라벨 인코딩 수행 (구 제외)
for column in seoul.select_dtypes(include=['object']).columns:
    if column != '구':
        le = LabelEncoder()
        seoul[column] = le.fit_transform(seoul[column])
        label_encoders[column] = le
        original_values[column] = le.classes_

# '구' 칼럼 라벨 인코딩
label_encoder_gu = LabelEncoder()
seoul['구'] = label_encoder_gu.fit_transform(seoul['구'])
label_encoders['구'] = label_encoder_gu
original_values['구'] = label_encoder_gu.classes_

# 라벨 인코더와 원래 값을 저장
with open('label_encoders_lgbm.pkl', 'wb') as le_file:
    pickle.dump(label_encoders, le_file)

with open('original_values_lgbm.pkl', 'wb') as ov_file:
    pickle.dump(original_values, ov_file)

print("라벨 인코더와 오리지널 값이 저장되었습니다.")

라벨 인코더와 오리지널 값이 저장되었습니다.


In [13]:
label_encoders

{'공장구분': LabelEncoder(),
 '설립구분': LabelEncoder(),
 '입주형태': LabelEncoder(),
 '보유구분': LabelEncoder(),
 '등록구분': LabelEncoder(),
 '공장규모': LabelEncoder(),
 '용도지역': LabelEncoder(),
 '지목': LabelEncoder(),
 '지식산업센터명': LabelEncoder(),
 '대표업종': LabelEncoder(),
 '구': LabelEncoder()}

In [14]:
original_values

{'공장구분': array(['개별', '계획'], dtype=object),
 '설립구분': array(['국가산업단지', '일반', '일반산업단지', '지식산업센터', '창업'], dtype=object),
 '입주형태': array([' ', '경매', '공장임대', '기타', '분양', '양수도', '일반건축물', '일반토지', '임대사업자',
        '해당없음'], dtype=object),
 '보유구분': array(['임대', '자가'], dtype=object),
 '등록구분': array(['등록변경', '부분등록', '신규등록', '완료신고'], dtype=object),
 '공장규모': array(['대기업', '소기업', '중기업'], dtype=object),
 '용도지역': array([' ', '관리지역/계획관리지역/계획관리지역/', '관리지역/계획관리지역/계획관리지역/미관지구',
        '도시지역  도시지역  준공업지역   ', '도시지역/공업지역/공업지역/', '도시지역/공업지역/일반공업지역/',
        '도시지역/공업지역/일반공업지역/기타지역', '도시지역/공업지역/전용공업지역/',
        '도시지역/공업지역/전용공업지역/기타지역', '도시지역/공업지역/준공업지역/',
        '도시지역/공업지역/준공업지역/개발지구', '도시지역/공업지역/준공업지역/개발진흥지구',
        '도시지역/공업지역/준공업지역/고도지구', '도시지역/공업지역/준공업지역/기타지역',
        '도시지역/공업지역/준공업지역/미관지구', '도시지역/공업지역/준공업지역/방화지구',
        '도시지역/공업지역/준공업지역/시설보호지구', '도시지역/공업지역/준공업지역/지구단위1종수립지구',
        '도시지역/공업지역/준공업지역/지구단위2종수립지구', '도시지역/녹지지역/보전녹지지역/',
        '도시지역/녹지지역/생산녹지지역/', '도시지역/녹지지역/자연녹지지역/', '도시지역/녹지지역/자연녹지지

In [13]:
seoul

,공장구분,설립구분,입주형태,보유구분,등록구분,남자종업원,여자종업원,외국인남자종업원,외국인여자종업원,종업원합계,공장규모,용도지역,지목,용지면적,제조시설면적,부대시설면적,건축면적,지식산업센터명,대표업종,구
0,0,1,9,0,2,0,0,0,0,0,1,57,2,238,57,69,126,223,105,22
1,0,1,9,0,0,5,2,0,0,7,1,78,2,157,79,78,157,223,396,22
2,0,1,9,1,0,12,12,0,0,24,1,78,2,160,92,67,160,223,422,22
3,0,1,9,0,2,0,0,0,0,0,1,59,2,280,35,88,123,223,105,22
4,0,1,9,0,0,7,2,0,0,9,1,34,2,122,159,18,177,223,108,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11628,0,1,6,0,0,1,4,0,0,5,1,58,2,104,104,0,104,223,68,1
11629,0,1,9,1,0,0,0,0,0,0,1,78,2,164,110,53,164,223,384,1
11630,0,1,6,0,0,7,0,0,0,7,1,58,2,125,114,11,125,223,77,1
11631,0,1,6,0,2,2,3,0,0,5,1,74,2,104,104,0,104,223,42,1


In [15]:
seoul.to_csv('./seoul_encoding_data.csv', index=False)

In [14]:
X = seoul.drop(columns=['구'])
y = seoul['구']

In [15]:
print(X.shape, y.shape)

(11633, 19) (11633,)


In [16]:
X

,공장구분,설립구분,입주형태,보유구분,등록구분,남자종업원,여자종업원,외국인남자종업원,외국인여자종업원,종업원합계,공장규모,용도지역,지목,용지면적,제조시설면적,부대시설면적,건축면적,지식산업센터명,대표업종
0,0,1,9,0,2,0,0,0,0,0,1,57,2,238,57,69,126,223,105
1,0,1,9,0,0,5,2,0,0,7,1,78,2,157,79,78,157,223,396
2,0,1,9,1,0,12,12,0,0,24,1,78,2,160,92,67,160,223,422
3,0,1,9,0,2,0,0,0,0,0,1,59,2,280,35,88,123,223,105
4,0,1,9,0,0,7,2,0,0,9,1,34,2,122,159,18,177,223,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11628,0,1,6,0,0,1,4,0,0,5,1,58,2,104,104,0,104,223,68
11629,0,1,9,1,0,0,0,0,0,0,1,78,2,164,110,53,164,223,384
11630,0,1,6,0,0,7,0,0,0,7,1,58,2,125,114,11,125,223,77
11631,0,1,6,0,2,2,3,0,0,5,1,74,2,104,104,0,104,223,42


In [17]:
y.value_counts()

7     3455
15    1357
6     1215
23    1190
19     845
3      642
17     428
24     341
10     265
0      228
14     208
5      174
22     166
9      155
1      129
18     108
8      105
16     103
12     102
21      94
2       89
20      77
4       65
13      52
11      40
Name: 구, dtype: int64

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(7910, 19) (1978, 19) (1745, 19) (7910,) (1978,) (1745,)


In [19]:
## label encoding
from sklearn.preprocessing import LabelEncoder
categorical_features = list(X_train.dtypes[X_train.dtypes == 'object'.index])

# 라벨 인코딩 호기화
le = LabelEncoder()

# Label encoding 적용
for feature in categorical_features:
    X_train[feature] = le.fit_transform(X_train[feature])
    X_val[feature] = le.fit_transform(X_val[feature])
    X_test[feature] = le.fit_transform(X_test[feature])

In [21]:
print(X_train.info(), X.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7910 entries, 2548 to 5416
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   공장구분      7910 non-null   int32
 1   설립구분      7910 non-null   int32
 2   입주형태      7910 non-null   int32
 3   보유구분      7910 non-null   int32
 4   등록구분      7910 non-null   int32
 5   남자종업원     7910 non-null   int32
 6   여자종업원     7910 non-null   int32
 7   외국인남자종업원  7910 non-null   int32
 8   외국인여자종업원  7910 non-null   int32
 9   종업원합계     7910 non-null   int32
 10  공장규모      7910 non-null   int32
 11  용도지역      7910 non-null   int32
 12  지목        7910 non-null   int32
 13  용지면적      7910 non-null   int32
 14  제조시설면적    7910 non-null   int32
 15  부대시설면적    7910 non-null   int32
 16  건축면적      7910 non-null   int32
 17  지식산업센터명   7910 non-null   int32
 18  대표업종      7910 non-null   int32
dtypes: int32(19)
memory usage: 648.9 KB
None 공장구분        0
설립구분        0
입주형태        0
보유구분        0
등록구분       

In [22]:
X_train

,공장구분,설립구분,입주형태,보유구분,등록구분,남자종업원,여자종업원,외국인남자종업원,외국인여자종업원,종업원합계,공장규모,용도지역,지목,용지면적,제조시설면적,부대시설면적,건축면적,지식산업센터명,대표업종
2548,0,1,9,1,2,10,1,2,0,13,1,9,2,495,245,54,299,223,165
5896,1,0,5,1,3,32,6,1,0,39,1,9,0,208,816,636,1452,96,250
2660,0,1,0,0,0,5,12,0,0,17,1,11,2,904,72,26,98,223,65
8386,1,3,5,1,3,2,9,0,0,11,1,9,0,66,271,175,446,172,62
3536,0,1,6,0,2,4,0,0,0,4,1,58,2,0,131,160,292,223,268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4166,0,1,6,0,0,1,2,0,0,3,1,1,2,170,22,148,170,223,39
9825,0,1,9,0,3,24,1,0,0,25,1,9,0,601,412,220,632,223,347
4383,0,1,9,1,3,11,2,0,0,13,1,9,2,359,0,110,110,223,229
1315,0,1,6,0,2,1,1,0,0,2,1,34,0,0,65,0,65,69,64


In [23]:
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import os

In [24]:
# 결과를 CSV 파일에 저장하는 콜백 함수
def save_trial_to_csv(study, trial):
    header = ['Trial Number', 'Classifier', 'Params', 'Value']
    file_exists = os.path.isfile('optuna_trials.csv')
    
    with open('optuna_trials.csv', mode='a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(header)
        
        params = trial.params
        classifier = params.pop('classifier')
        writer.writerow([trial.number, classifier, params, trial.value])


In [27]:
# Optuna 목적 함수
def objective(trial, X_train, y_train, X_val, y_val):
    classifier_name = trial.suggest_categorical('classifier', ['XGBClassifier', 'LGBMClassifier'])

    if classifier_name == 'XGBClassifier':
        params = {
            'max_depth': trial.suggest_int('max_depth', 1, 9),
            'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 10),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 10)
        }
        model = XGBClassifier(**params, use_label_encoder=False)
    else:
        params = {
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
            'min_split_gain': trial.suggest_float('min_split_gain', 0, 10)
        }
        model = LGBMClassifier(**params)

    if classifier_name == 'XGBClassifier':
        model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  eval_metric='mlogloss',
                  early_stopping_rounds=5,
                  verbose=False)
    else:
        model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  eval_metric='multi_logloss')
    
    score = model.score(X_val, y_val)
    return score


In [28]:
# 하이퍼파라미터 최적화
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=100, callbacks=[save_trial_to_csv])

# 최적의 하이퍼파라미터 출력
best_params = study.best_trial.params
print("Best trial parameters:", best_params)

[I 2024-06-18 21:49:20,923] A new study created in memory with name: no-name-a9457c0a-8baf-4bd9-906c-353ca23124b7
c:\Users\vivid\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\vivid\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2024-06-18 21:49:23,641] Trial 0 finished with value: 0.5556117290192113 and parameters: {'classifier': 'XGBClassifier', 'max_depth': 3, 'learning_rate': 0.04293418170459775, 'n_estimators': 282, 'subsample': 0.6402395285555496, 'colsample_bytree': 0.6137015255959584, 'gamma': 3.310049710908837, 'min_child_weight': 3, 'reg_alpha': 8.588389442483212, 'reg_lambda': 8.19750468969

[1]	valid_0's multi_logloss: 2.48678
[2]	valid_0's multi_logloss: 2.48446
[3]	valid_0's multi_logloss: 2.48205
[4]	valid_0's multi_logloss: 2.47979
[5]	valid_0's multi_logloss: 2.47721
[6]	valid_0's multi_logloss: 2.47488
[7]	valid_0's multi_logloss: 2.47248
[8]	valid_0's multi_logloss: 2.47002
[9]	valid_0's multi_logloss: 2.46773
[10]	valid_0's multi_logloss: 2.46539
[11]	valid_0's multi_logloss: 2.46317
[12]	valid_0's multi_logloss: 2.46093
[13]	valid_0's multi_logloss: 2.45861
[14]	valid_0's multi_logloss: 2.45639
[15]	valid_0's multi_logloss: 2.45405
[16]	valid_0's multi_logloss: 2.45176
[17]	valid_0's multi_logloss: 2.44947
[18]	valid_0's multi_logloss: 2.44754
[19]	valid_0's multi_logloss: 2.44532
[20]	valid_0's multi_logloss: 2.44304
[21]	valid_0's multi_logloss: 2.44096
[22]	valid_0's multi_logloss: 2.43854
[23]	valid_0's multi_logloss: 2.43633
[24]	valid_0's multi_logloss: 2.43409
[25]	valid_0's multi_logloss: 2.43194
[26]	valid_0's multi_logloss: 2.42983
[27]	valid_0's multi_

[I 2024-06-18 21:49:29,065] Trial 1 finished with value: 0.4418604651162791 and parameters: {'classifier': 'LGBMClassifier', 'num_leaves': 72, 'learning_rate': 0.0011662629077127532, 'n_estimators': 437, 'min_child_samples': 71, 'subsample': 0.7269780854236079, 'colsample_bytree': 0.8426739968580796, 'reg_alpha': 8.3385645736952, 'reg_lambda': 8.758100064256615, 'min_split_gain': 1.696135630614064}. Best is trial 0 with value: 0.5556117290192113.


[1]	valid_0's multi_logloss: 2.48894
[2]	valid_0's multi_logloss: 2.48878
[3]	valid_0's multi_logloss: 2.48861
[4]	valid_0's multi_logloss: 2.48846
[5]	valid_0's multi_logloss: 2.48831
[6]	valid_0's multi_logloss: 2.48813
[7]	valid_0's multi_logloss: 2.48795
[8]	valid_0's multi_logloss: 2.48777
[9]	valid_0's multi_logloss: 2.4876
[10]	valid_0's multi_logloss: 2.48742
[11]	valid_0's multi_logloss: 2.48729
[12]	valid_0's multi_logloss: 2.48711
[13]	valid_0's multi_logloss: 2.48694
[14]	valid_0's multi_logloss: 2.48677
[15]	valid_0's multi_logloss: 2.4866
[16]	valid_0's multi_logloss: 2.48644
[17]	valid_0's multi_logloss: 2.48626
[18]	valid_0's multi_logloss: 2.48614
[19]	valid_0's multi_logloss: 2.48597
[20]	valid_0's multi_logloss: 2.48581
[21]	valid_0's multi_logloss: 2.48567
[22]	valid_0's multi_logloss: 2.4855
[23]	valid_0's multi_logloss: 2.48534
[24]	valid_0's multi_logloss: 2.48516
[25]	valid_0's multi_logloss: 2.48499
[26]	valid_0's multi_logloss: 2.48482
[27]	valid_0's multi_log

[I 2024-06-18 21:49:32,234] Trial 2 finished with value: 0.29575328614762386 and parameters: {'classifier': 'LGBMClassifier', 'num_leaves': 252, 'learning_rate': 8.641379707852157e-05, 'n_estimators': 268, 'min_child_samples': 91, 'subsample': 0.6802287717972023, 'colsample_bytree': 0.6664455362184868, 'reg_alpha': 6.571288944194289, 'reg_lambda': 7.811975932076916, 'min_split_gain': 3.121613616207611}. Best is trial 0 with value: 0.5556117290192113.
c:\Users\vivid\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\vivid\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2024-06-18 21:49:37,904] Trial 3 finis

...

[1]	valid_0's multi_logloss: 2.46498
[2]	valid_0's multi_logloss: 2.43896
[3]	valid_0's multi_logloss: 2.41533
[4]	valid_0's multi_logloss: 2.39497
[5]	valid_0's multi_logloss: 2.37312
[6]	valid_0's multi_logloss: 2.34951
[7]	valid_0's multi_logloss: 2.32676
[8]	valid_0's multi_logloss: 2.30502
[9]	valid_0's multi_logloss: 2.28408
[10]	valid_0's multi_logloss: 2.2632
[11]	valid_0's multi_logloss: 2.24599
[12]	valid_0's multi_logloss: 2.22683
[13]	valid_0's multi_logloss: 2.20789
[14]	valid_0's multi_logloss: 2.19067
[15]	valid_0's multi_logloss: 2.17348
[16]	valid_0's multi_logloss: 2.15745
[17]	valid_0's multi_logloss: 2.14064
[18]	valid_0's multi_logloss: 2.12791
[19]	valid_0's multi_logloss: 2.11247
[20]	valid_0's multi_logloss: 2.09781
[21]	valid_0's multi_logloss: 2.08517
[22]	valid_0's multi_logloss: 2.06997
[23]	valid_0's multi_logloss: 2.05698
[24]	valid_0's multi_logloss: 2.04214
[25]	valid_0's multi_logloss: 2.02837
[26]	valid_0's multi_logloss: 2.01509
[27]	valid_0's multi_l

[I 2024-06-18 21:55:37,069] Trial 97 finished with value: 0.6112234580384226 and parameters: {'classifier': 'LGBMClassifier', 'num_leaves': 88, 'learning_rate': 0.010763894442553035, 'n_estimators': 153, 'min_child_samples': 87, 'subsample': 0.5609412137254504, 'colsample_bytree': 0.6615133395066995, 'reg_alpha': 1.961750453133281, 'reg_lambda': 1.8222461192952788, 'min_split_gain': 1.0122195487920926}. Best is trial 92 with value: 0.6880687563195147.
c:\Users\vivid\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\vivid\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2024-06-18 21:55:40,944] Trial 98 fin

[1]	valid_0's multi_logloss: 2.27049
[2]	valid_0's multi_logloss: 2.11044
[3]	valid_0's multi_logloss: 1.97653
[4]	valid_0's multi_logloss: 1.86733
[5]	valid_0's multi_logloss: 1.77718
[6]	valid_0's multi_logloss: 1.70431
[7]	valid_0's multi_logloss: 1.64174
[8]	valid_0's multi_logloss: 1.58293
[9]	valid_0's multi_logloss: 1.53797
[10]	valid_0's multi_logloss: 1.50261
[11]	valid_0's multi_logloss: 1.46374
[12]	valid_0's multi_logloss: 1.43542
[13]	valid_0's multi_logloss: 1.40683
[14]	valid_0's multi_logloss: 1.38218
[15]	valid_0's multi_logloss: 1.36324
[16]	valid_0's multi_logloss: 1.34398
[17]	valid_0's multi_logloss: 1.32231
[18]	valid_0's multi_logloss: 1.30571
[19]	valid_0's multi_logloss: 1.28812
[20]	valid_0's multi_logloss: 1.27233
[21]	valid_0's multi_logloss: 1.26127
[22]	valid_0's multi_logloss: 1.2491
[23]	valid_0's multi_logloss: 1.23651
[24]	valid_0's multi_logloss: 1.22887
[25]	valid_0's multi_logloss: 1.22017
[26]	valid_0's multi_logloss: 1.21009
[27]	valid_0's multi_l

[I 2024-06-18 21:55:44,088] Trial 99 finished with value: 0.6830131445904954 and parameters: {'classifier': 'LGBMClassifier', 'num_leaves': 99, 'learning_rate': 0.0868399738548613, 'n_estimators': 472, 'min_child_samples': 30, 'subsample': 0.5482702077986829, 'colsample_bytree': 0.6753228392273224, 'reg_alpha': 1.0613236856078936, 'reg_lambda': 2.811110077344856, 'min_split_gain': 0.2694181807728545}. Best is trial 92 with value: 0.6880687563195147.


Best trial parameters: {'classifier': 'LGBMClassifier', 'num_leaves': 97, 'learning_rate': 0.06291686692582928, 'n_estimators': 471, 'min_child_samples': 69, 'subsample': 0.5148941824713373, 'colsample_bytree': 0.6947463127096882, 'reg_alpha': 0.40420315890792746, 'reg_lambda': 3.172929043100316, 'min_split_gain': 0.33450025083007257}


In [29]:
# 최적의 하이퍼파라미터로 모델 훈련
if best_params['classifier'] == 'XGBClassifier':
    best_model = XGBClassifier(
        max_depth=best_params['max_depth'],
        learning_rate=best_params['learning_rate'],
        n_estimators=best_params['n_estimators'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        gamma=best_params['gamma'],
        min_child_weight=best_params['min_child_weight'],
        reg_alpha=best_params['reg_alpha'],
        reg_lambda=best_params['reg_lambda'],
        use_label_encoder=False
    )
else:
    best_model = LGBMClassifier(
        num_leaves=best_params['num_leaves'],
        learning_rate=best_params['learning_rate'],
        n_estimators=best_params['n_estimators'],
        min_child_samples=best_params['min_child_samples'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        reg_alpha=best_params['reg_alpha'],
        reg_lambda=best_params['reg_lambda'],
        min_split_gain=best_params['min_split_gain']
    )

# 최적의 모델을 사용하여 최종 훈련 및 평가
if best_params['classifier'] == 'XGBClassifier':
    best_model.fit(X_train, y_train,
                   eval_set=[(X_val, y_val)],
                   eval_metric='mlogloss',
                   early_stopping_rounds=10,
                   verbose=False)
else:
    best_model.fit(X_train, y_train,
                   eval_set=[(X_val, y_val)],
                   eval_metric='multi_logloss')

[1]	valid_0's multi_logloss: 2.33823
[2]	valid_0's multi_logloss: 2.21025
[3]	valid_0's multi_logloss: 2.10883
[4]	valid_0's multi_logloss: 2.0284
[5]	valid_0's multi_logloss: 1.94923
[6]	valid_0's multi_logloss: 1.87803
[7]	valid_0's multi_logloss: 1.81591
[8]	valid_0's multi_logloss: 1.76483
[9]	valid_0's multi_logloss: 1.71632
[10]	valid_0's multi_logloss: 1.67406
[11]	valid_0's multi_logloss: 1.63516
[12]	valid_0's multi_logloss: 1.60078
[13]	valid_0's multi_logloss: 1.56721
[14]	valid_0's multi_logloss: 1.53718
[15]	valid_0's multi_logloss: 1.51029
[16]	valid_0's multi_logloss: 1.48666
[17]	valid_0's multi_logloss: 1.46403
[18]	valid_0's multi_logloss: 1.4438
[19]	valid_0's multi_logloss: 1.42479
[20]	valid_0's multi_logloss: 1.40552
[21]	valid_0's multi_logloss: 1.38905
[22]	valid_0's multi_logloss: 1.37095
[23]	valid_0's multi_logloss: 1.35541
[24]	valid_0's multi_logloss: 1.33983
[25]	valid_0's multi_logloss: 1.32499
[26]	valid_0's multi_logloss: 1.31106
[27]	valid_0's multi_lo

In [38]:
# 모델 딕셔너리 초기화 및 최적 모델 추가
models = {'BestModel': best_model}

In [39]:
models

{'BestModel': LGBMClassifier(colsample_bytree=0.6947463127096882,
                learning_rate=0.06291686692582928, min_child_samples=69,
                min_split_gain=0.33450025083007257, n_estimators=471,
                num_leaves=97, reg_alpha=0.40420315890792746,
                reg_lambda=3.172929043100316, subsample=0.5148941824713373)}

In [40]:
#train, val 시각화 함수
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 결과 시각화 함수 확장
def evaluate_models(models, X_train, y_train, X_val, y_val):
    max_name_length = max([len(name) for name in models.keys()]) + 1  # 최대 모델 이름 길이 계산

    print("Model Performance Evaluation:\n")
    header = f"{'Model'.ljust(max_name_length)}\tTrain Accuracy\tVal Accuracy\tTrain F1\tVal F1\tTrain Precision\tVal Precision\tTrain Recall\tVal Recall"
    print(header)
    print("-" * len(header))

    for name, model in models.items():
        pred_train = model.predict(X_train)
        pred_val = model.predict(X_val)

        accuracy_train = accuracy_score(y_train, pred_train)
        accuracy_val = accuracy_score(y_val, pred_val)
        f1_train = f1_score(y_train, pred_train, average='macro')
        f1_val = f1_score(y_val, pred_val, average='macro')
        precision_train = precision_score(y_train, pred_train, average='macro')
        precision_val = precision_score(y_val, pred_val, average='macro')
        recall_train = recall_score(y_train, pred_train, average='macro')
        recall_val = recall_score(y_val, pred_val, average='macro')

        print(f"{name.ljust(max_name_length)}\t{accuracy_train:.4f}\t\t{accuracy_val:.4f}\t\t{f1_train:.4f}\t\t{f1_val:.4f}\t\t{precision_train:.4f}\t\t{precision_val:.4f}\t\t{recall_train:.4f}\t\t{recall_val:.4f}")


In [41]:
# 모델 성능 평가
evaluate_models(models, X_train, y_train, X_val, y_val)

Model Performance Evaluation:

Model     	Train Accuracy	Val Accuracy	Train F1	Val F1	Train Precision	Val Precision	Train Recall	Val Recall
------------------------------------------------------------------------------------------------------------
BestModel 	0.9029		0.6881		0.8131		0.3652		0.8644		0.3786		0.7832		0.3649


c:\Users\vivid\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
# 테스트 데이터로 모델 평가
test_accuracy = best_model.score(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.6814


In [31]:
# 교차 검증으로 모델 평가
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(best_model, X, y, cv=cv, scoring='accuracy')
print(f"Cross-Validation Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")

Cross-Validation Accuracy: 0.6990 ± 0.0039


In [32]:
# 교차 검증으로 모델 평가
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(best_model, X, y, cv=cv, scoring='accuracy')
print(f"Cross-Validation Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")

Cross-Validation Accuracy: 0.7037 ± 0.0083


In [33]:
# 교차 검증으로 모델 평가
cv = StratifiedKFold(n_splits=15, shuffle=True, random_state=42)
scores = cross_val_score(best_model, X, y, cv=cv, scoring='accuracy')
print(f"Cross-Validation Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")

Cross-Validation Accuracy: 0.7040 ± 0.0075


In [34]:
# 교차 검증으로 모델 평가
cv = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
scores = cross_val_score(best_model, X, y, cv=cv, scoring='accuracy')
print(f"Cross-Validation Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")

Cross-Validation Accuracy: 0.7030 ± 0.0158
